# Klavis Sandbox

This notebook demonstrates how to use Klavis Sandbox and MCP Servers


In [ ]:
%pip install -q klavis python-dotenv langchain langchain-mcp-adapters langchain-openai

In [ ]:
import json
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from klavis import Klavis, SandboxMcpServer

load_dotenv()

CURRENT_DIR = os.path.dirname(os.path.abspath("__file__")) if os.path.dirname(os.path.abspath("__file__")) else os.getcwd()
LOCAL_MCP_SERVERS_PATH = os.path.abspath(os.path.join(CURRENT_DIR, "..", "..", "mcp_servers", "local"))

klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

print(f"Current Directory: {CURRENT_DIR}")
print(f"MCP Servers Local Path: {LOCAL_MCP_SERVERS_PATH}")

### Step 1: Create Sandboxes environment for Gmail and Snowflake

In [ ]:
# Create Gmail sandbox
gmail_sandbox = klavis_client.sandbox.create_sandbox(
    server_name="gmail",
)

snowflake_sandbox = klavis_client.sandbox.create_sandbox(
    server_name=SandboxMcpServer.SNOWFLAKE,
)

print(f"Gmail Sandbox: {gmail_sandbox}")
print(f"Snowflake Sandbox: {snowflake_sandbox}")

In [ ]:
# check existing environment in gmail sandbox
response = klavis_client.sandbox.dump_sandbox(
    sandbox_id=gmail_sandbox.sandbox_id,
)

print(json.dumps(response.data, indent=2))

In [ ]:
# check existing environment in snowflake sandbox
response = klavis_client.sandbox.dump_sandbox(
    sandbox_id=snowflake_sandbox.sandbox_id,
)

print(json.dumps(response.data, indent=2))

### Step 2: Initialize Snowflake Sandbox with Data

In [ ]:
snowflake_json_path = os.path.join(CURRENT_DIR, "snowflake.json")

with open(snowflake_json_path, "r") as f:
    snowflake_data = json.load(f)
    
klavis_client.sandbox.initialize_sandbox(
    sandbox_id=snowflake_sandbox.sandbox_id,
    databases=snowflake_data["databases"],
)

In [ ]:
# dump data 
response = klavis_client.sandbox.dump_sandbox(
    sandbox_id=snowflake_sandbox.sandbox_id,
)

print(json.dumps(response.data, indent=2))

### Step 3: Setup MCP Client, AI Agent and kick off task

In [ ]:
# Create MCP client with remote and local MCP Servers
mcp_client = MultiServerMCPClient({
    "snowflake": {
        "transport": "streamable_http",
        "url": snowflake_sandbox.server_url,
    },
    "gmail": {
        "transport": "streamable_http",
        "url": gmail_sandbox.server_url,
    },
    "filesystem": {
        "transport": "stdio",
        "command": "npx",
        "args": [
            "-y",
            os.path.join(LOCAL_MCP_SERVERS_PATH, "filesystem"),
            CURRENT_DIR
        ],
        "env": dict(os.environ)
    },
    "pdf-tools": {
        "transport": "stdio",
        "command": "uvx",
        "args": [
            "--from",
            os.path.join(LOCAL_MCP_SERVERS_PATH, "pdf-tools"),
            "pdf-tools-mcp",
            "--workspace_path",
            CURRENT_DIR,
            "--tempfile_dir",
            CURRENT_DIR
        ],
        "env": dict(os.environ)
    },
})

tools = await mcp_client.get_tools()
llm = ChatOpenAI(model="gpt-5-2025-08-07")

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=(
        "You are a helpful assistant that can use MCP tools. "
    ),
)

In [ ]:
# print tools
for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}")

In [ ]:
user_message = "run the query to give me the first 10 rows of table SLA_MONITOR"

# Run AI Agent with streaming
print("🚀 Starting agent...\n")

async for event in agent.astream_events(
    {"messages": [{"role": "user", "content": user_message}]},
    version="v2",
):
    kind = event["event"]
    
    # AI is thinking/responding
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(content, end="", flush=True)
    
    # Tool is being called
    elif kind == "on_tool_start":
        tool_name = event["name"]
        tool_input = event["data"].get("input", {})
        print(f"\n\n🔧 Calling tool: {tool_name}")
        print(f"   Input: {tool_input}")
    
    # Tool finished
    elif kind == "on_tool_end":
        tool_name = event["name"]
        output = event["data"].get("output", "")
        print(f"✅ Tool {tool_name} completed")
        # Print first 200 chars of output
        output_str = str(output)[:200]
        print(f"   Output preview: {output_str}...")

print("\n\n✨ Agent finished!")

### Step 5: Cleanup - Delete Sandboxes

In [ ]:
klavis_client.sandbox.delete_sandbox(
    server_name=SandboxMcpServer.GMAIL,
    sandbox_id=gmail_sandbox.sandbox_id,
)

klavis_client.sandbox.delete_sandbox(
    server_name=SandboxMcpServer.SNOWFLAKE,
    sandbox_id=snowflake_sandbox.sandbox_id,
)